In [1]:
"""
1) Survey: SurveyID|Description
2) Question: QuestionID|QuestionText
3) Answer: SurveyID|UserID|QuestionID|AnswerText

SurveyID are simply year, ie. 2014, 2016, 2017, 2018, 2019
"""

hello world


In [53]:
query1 = """
SELECT *
FROM Survey;
"""
query2 = """
SELECT *
FROM Question;
"""
query3 = """
SELECT *
FROM Answer;
"""
query4 = """
SELECT Answer.AnswerText, Answer.SurveyID, Answer.UserID,
        Answer.QuestionID, Question.questiontext QuestionText,
        Survey.Description
FROM Answer
JOIN Question
ON Answer.QuestionID = Question.QuestionID
JOIN Survey
ON Answer.SurveyID = Survey.SurveyID
"""

In [54]:
import sqlite3
import pandas as pd

location = r"D:\IT_projects\Turing_Colledge\Modul2\Sprint1\project\mental_health.sqlite"
con = sqlite3.connect(location)

#1-4, 2-105, 3-236898

In [55]:
pd.read_sql_query(query4, con)

,AnswerText,SurveyID,UserID,QuestionID,QuestionText,Description
0,37,2014,1,1,What is your age?,mental health survey for 2014
1,44,2014,2,1,What is your age?,mental health survey for 2014
2,32,2014,3,1,What is your age?,mental health survey for 2014
3,31,2014,4,1,What is your age?,mental health survey for 2014
4,31,2014,5,1,What is your age?,mental health survey for 2014
...,...,...,...,...,...,...
236893,Other,2016,2689,117,Which of the following best describes your wor...,mental health survey for 2016
236894,Support,2016,2690,117,Which of the following best describes your wor...,mental health survey for 2016
236895,Back-end Developer,2016,2691,117,Which of the following best describes your wor...,mental health survey for 2016
236896,DevOps/SysAdmin,2016,2692,117,Which of the following best describes your wor...,mental health survey for 2016
